<a href="https://colab.research.google.com/github/kohanpour1/Ehsan/blob/main/prophet_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet

     |████████████████████████████████| 6.3 MB 6.8 MB/s 


In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [3]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'

eth_df = yf.download('ETH-USD',start_date, today)

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-10-07,"$3,576.81","$3,650.01","$3,479.91","$3,587.97","$3,587.97",19090322927
2021-10-08,"$3,587.83","$3,667.96","$3,547.83","$3,563.76","$3,563.76",16222029488
2021-10-09,"$3,560.00","$3,628.24","$3,544.64","$3,575.72","$3,575.72",12707036942
2021-10-10,"$3,575.02","$3,603.24","$3,414.97","$3,425.85","$3,425.85",16171746693
2021-10-11,"$3,385.76","$3,619.50","$3,385.76","$3,576.89","$3,576.89",18700105728


In [4]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2107 entries, 2016-01-01 to 2021-10-11
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2107 non-null   float64
 1   High       2107 non-null   float64
 2   Low        2107 non-null   float64
 3   Close      2107 non-null   float64
 4   Adj Close  2107 non-null   float64
 5   Volume     2107 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 115.2 KB


In [5]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [6]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [7]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [8]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds", 
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [9]:
df.tail()

,ds,y
2102,2021-10-07,"$3,576.81"
2103,2021-10-08,"$3,587.83"
2104,2021-10-09,"$3,560.00"
2105,2021-10-10,"$3,575.02"
2106,2021-10-11,"$3,385.76"


In [10]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [11]:
m = Prophet(
    seasonality_mode="multiplicative" 
)

m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2467,2022-10-07
2468,2022-10-08
2469,2022-10-09
2470,2022-10-10
2471,2022-10-11


In [13]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2467,2022-10-07,"$5,892.90","$5,170.82","$6,581.91"
2468,2022-10-08,"$5,885.15","$5,144.07","$6,612.31"
2469,2022-10-09,"$5,894.59","$5,174.05","$6,599.19"
2470,2022-10-10,"$5,903.51","$5,164.81","$6,587.44"
2471,2022-10-11,"$5,844.84","$5,114.06","$6,560.30"


In [14]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

3201.2793656979934

In [15]:
plot_plotly(m, forecast)

In [16]:
plot_components_plotly(m, forecast)